### Stratégie :
On récupère le point de départ, on le région growth.
On fait une recherche sur la couche du dessous sur le centre trouvé précédemment puis en région growing sur les plus proches voisins jusqu'à trouver un point d'intensité similaire +-5% puis on le region growth. On limite le region growing à la taille de la tache trouvée sur la précédente couche + 1 voisin
Si on ne trouve rien dans cette zone, on met un False dans la liste et on descend d'une couche avec une tolérance d'intensité de +1% et d'un voisin supplémentaire 

In [1]:
import SimpleITK as sitk
import itk
import itkwidgets 
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np
import ipywidgets as widgets
import cv2
from tqdm.notebook import tqdm
from napari.utils.colormaps import colormap_utils as cu

In [2]:
class RunningAverage:
    def __init__(self,sum=0,count=0,last=None):
        self.sum = sum
        self.count = count
        self.last = last
        

    def add_number(self, number):
        self.sum += number
        self.last = number
        self.count += 1

    def get_average(self):
        if self.count == 0:
            return 0 
        return self.sum / self.count
    
    def split(self):
        self.sum /= 4
        self.count /= 2
        return self
    
    def get_last(self):
        return self.last
    
    def copy(self):
        return RunningAverage(self.sum,self.count,self.last)
    
class NestedAverage:
    def __init__(self,numbers=list(),limit=10,average=0,count=0):
        self.limit = limit
        self.average = average
        self.count = count
        self.numbers = numbers
        if self.average == 0 and self.count == 0:
            self.numbers = []
        
        
        
    def add_number(self, number):
        self.numbers.append(number)
        if self.count > self.limit:
            self.numbers.pop(0)
        else:
            self.count += 1
        self.average = sum(self.numbers)/self.count

    def get_average(self):
        return self.average
    
    def copy(self):
        return NestedAverage(self.numbers,self.limit,self.average,self.count)

class NetworkManager:
    def __init__(self,depth):
        self.last_branch_id = 0
        self.network = list()
        for i in range(depth):
            self.network.append(list())

    def get_new_branch(self):
        self.last_branch_id = self.last_branch_id+1
        return self.last_branch_id
    
    def append_to_network(self,depth,branch_id,contour,predicted=False):
        value = np.append(self.network[depth],NetworkManagerArea(branch_id,contour,predicted))
        self.network[depth] = value
    
    def get_branch(self,branch_id):
        branch = list()
        offset = 0
        for i in reversed(range(len(self.network))):
            area = next((area for area in self.network[i] if area.branch_id == branch_id),None)
            if len(branch) == 0:
                offset = i
            if area != None:
                branch.append(area)
            else:
                if len(branch)>0:
                    break
        return branch,offset


    #Each subnetwork is a different image
    def generate3DImages(self,shape):
        generated_images = []
        for branch_id in range(1,self.last_branch_id+1):
            image = np.zeros(shape)
            branch,offset = self.get_branch(branch_id)
            for area in branch:
                for point in area.contour:
                    image[offset][point[0][1]][point[0][0]] = 255
                offset -= 1
            generated_images.append(image)
        return generated_images

class NetworkManagerArea:
    def __init__(self,branch_id,contour,predicted):
        self.branch_id = branch_id
        self.contour = contour
        self.predicted = predicted

    def __str__(self) -> str:
        return "(branch_id : "+str(self.branch_id)+", contour : "+str(self.contour)+", predicted : "+str(self.predicted)+")"



In [3]:

def regionSearch(image,point,searchRadius,minValue):
    search_memory = np.zeros_like(image, dtype=np.uint8)
    stack = [point]
    while stack:
        x, y = stack.pop()
        if search_memory[y, x] == 0:
            # Vérifier si la valeur du pixel est dans la plage acceptable
            # if point == (265,208):
            #     print(x, y)
            #     print(point)
            if int(image[y, x]) > minValue: #and int(image[y, x]) < maxValue:
                return (x,y)
            else:
                search_memory[y, x] = 1
            # Ajouter les voisins à la pile
            if abs(y-point[1]) < searchRadius and abs(x-point[0]) < searchRadius:
                for i in range(-1, 2):
                    for j in range(-1, 2):
                        if 0 <= x + i < image.shape[1] and 0 <= y + j < image.shape[0]:
                            stack.append((x + i, y + j))
    print("Radius was "+str(searchRadius)+" origin was "+str(point) +" and value was "+str(minValue))
    return (-1,-1)

def regionGrowing(image,point,nested_intensity,alpha=0.60):
    segmented = np.zeros_like(image, dtype=np.uint8)

    # Initialiser la pile pour la croissance de la région
    stack = [point]

    # Récupérer la valeur du pixel du point de départ
    seed_value = nested_intensity.get_average()

    while stack:
        x, y = stack.pop()

        # Vérifier si le pixel est déjà visité
        if segmented[y, x] == 0:
            # Vérifier si la valeur du pixel est dans la plage acceptable
            if image[y, x] >= seed_value*alpha:
                # Ajouter le pixel à la région
                segmented[y, x] = 255

                # Ajouter les voisins à la pile
                for i in range(-1, 2):
                    for j in range(-1, 2):
                        if 0 <= x + i < image.shape[1] and 0 <= y + j < image.shape[0]:
                            stack.append((x + i, y + j))
    contours, _ = cv2.findContours(segmented, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    if len(contours)>0:
        return contours[0]
    else:
        print("Nothing found")
        return list()
    

In [4]:
def treeLookup(image,base_point,start=False,end=False,method="conic",alpha_search=0.6):
    #Processing default parameters
    if end == False:
        stop_index = 0
    else:
        stop_index = end

    if start == False:
        start_index = len(image)
    else:
        start_index = start
    mean_radius = RunningAverage()
    nested_intensity = NestedAverage()
    network_manager = NetworkManager(len(image))
    return subTreeLookup(network_manager,image,base_point,start_index,stop_index,method,alpha_search,mean_radius,nested_intensity,network_manager.get_new_branch())

def subTreeLookup(network_manager,image,base_point,start_index,stop_index,method,alpha_search,mean_radius,nested_intensity,branch_id):
    current_index = start_index
    current_pixel = base_point
    nested_intensity.add_number(image[start_index,base_point[1],base_point[0]])
    
    splitted_search = False
    successfull_split = False
    
    tube_contour = regionGrowing(image[current_index,:,:],current_pixel,nested_intensity)
    if len(tube_contour) == 0:
        return network_manager
    center, radius = cv2.minEnclosingCircle(tube_contour)
    current_pixel = (int(center[0]),int(center[1]))
    mean_radius.add_number(radius)
    network_manager.append_to_network(current_index,branch_id,tube_contour)
    if branch_id == 1:
        pbar = tqdm(total=start_index-stop_index)
    
    while (current_index > stop_index - 1):
        current_index = current_index - 1
        if branch_id == 1:
            pbar.update(1)
        if splitted_search:
            print("subtreeid "+str(branch_id)+" z-index "+str(current_index))
            if branch_id == 1:
                pbar.set_description("Searching for splitted in region %s" % current_index)
            last_radius = mean_radius.get_last()
            x_offset = int(last_radius/4)
            pixel_1 = (current_pixel[0]-x_offset,current_pixel[1])
            pixel_2 = (current_pixel[0]+x_offset,current_pixel[1])
            target_1 = regionSearch(image[current_index,:,:],pixel_1,(last_radius/2)-1,nested_intensity.get_average()*alpha_search)
            target_2 = regionSearch(image[current_index,:,:],pixel_2,(last_radius/2)-1,nested_intensity.get_average()*alpha_search)
            if target_1[0] != -1:
                print("target1 found "+str(target_1))
                foundTarget = target_1
            if target_2[0] != -1:
                #print("target2 found")
                subTreeLookup(network_manager,image,target_2,current_index-1,stop_index,method,alpha_search,mean_radius.split().copy(),nested_intensity.copy(),network_manager.get_new_branch())
                #TODO trouver un moyen de supprimer les sous reseaux trop court
                successfull_split = True
                # print(len(tube_network[-1]))
                # if len(tube_network[-1]) > 0:
                #     print(len(tube_network[-1][0]))
                #     print(len(tube_network[-1][0][0]))
                # if len(tube_network[-1]) > 20:
                #     successfull_split = True
                # else:
                #     continue
                    #tube_network.pop()
        else:
            #print("looking on xy:"+str(current_pixel)+" z:"+str(current_index)+" radius :"+str(radius))
            if branch_id == 1:
                pbar.set_description("Searching region %s" % current_index)
            foundTarget = regionSearch(image[current_index,:,:],current_pixel,radius,nested_intensity.get_average()*alpha_search)

        if foundTarget[0] != -1:
            if branch_id == 1:
                pbar.set_description("Growing region %s, radius %s" % (current_index,radius) )
            if splitted_search:
                tube_contour = regionGrowing(image[current_index,:,:],foundTarget,nested_intensity)
                if len(tube_contour) == 0:
                    print("target1 lead to end")
                    return network_manager
                center, radius = cv2.minEnclosingCircle(tube_contour)
            else:
                tube_contour = regionGrowing(image[current_index,:,:],foundTarget,nested_intensity)
                if len(tube_contour) == 0:
                    return network_manager
                center, radius = cv2.minEnclosingCircle(tube_contour)
            
            if not successfull_split and radius < mean_radius.get_average()*0.6:
                current_index = current_index + 1
                splitted_search = True
                print("splitted suspicion : "+str(radius)+" while 60% mean "+str(mean_radius.get_average()*0.6))
                continue
            
            if successfull_split:
                mean_radius.split()
                print("splitted appended")
            successfull_split = False
            splitted_search = False

            mean_radius.add_number(radius)
            current_pixel = (int(center[0]),int(center[1]))
            nested_intensity.add_number(image[current_index,current_pixel[1],current_pixel[0]])
            network_manager.append_to_network(current_index,branch_id,tube_contour)
        else:
            return network_manager
            break
            #Target not found 
            
    # image_with_contours = image.copy()
    # cv2.drawContours(image_with_contours, contours, -1, 255, 10)
    # plt.imshow(image_with_contours)
    pbar.close()
    return network_manager



In [5]:
#TODO if validated split, region 1 can't hover region2 during the next 20 images
image_array = np.load(Path("temp/preprocessed_image.npy"))
networkManager = treeLookup(image_array,(255,284),1531,300)

  0%|          | 0/1231 [00:00<?, ?it/s]

splitted suspicion : 5.72448205947876 while 60% mean 8.191431531621449
subtreeid 1 z-index 1263
target1 found (263, 208)
splitted suspicion : 4.301262855529785 while 60% mean 4.307023999179423
subtreeid 2 z-index 1149
target1 found (305, 211)
Nothing found
splitted appended
splitted suspicion : 1.414313554763794 while 60% mean 1.8317576522613639
subtreeid 2 z-index 1127
target1 found (320, 190)
Nothing found
splitted appended
Radius was 1.414313554763794 origin was (319, 191) and value was 45.27272727272727
splitted appended
splitted suspicion : 3.0504562854766846 while 60% mean 3.6263803605703595
subtreeid 1 z-index 1142
target1 found (311, 209)
splitted suspicion : 1.414313554763794 while 60% mean 2.17081821900275
subtreeid 5 z-index 1127
target1 found (320, 190)
Nothing found
splitted appended
Radius was 1.414313554763794 origin was (319, 191) and value was 43.581818181818186
splitted appended
splitted suspicion : 1.1181340217590332 while 60% mean 1.607075708830043
subtreeid 1 z-ind

In [10]:
# mask = np.zeros_like(image_array, dtype=np.uint8)
# mask2 = np.zeros_like(image_array, dtype=np.uint8)
# last_search_mask = np.zeros_like(image_array, dtype=np.uint8)
# last_search_cut = last_search_mask[1265]
# cv2.circle(last_search_cut,(266,205), 361, 255, 1)
# cv2.circle(last_search_cut,(266,205), 1, 255, 1)
# last_search_mask[1532-len(tube_network)] = last_search_cut
#print(image_array.shape)
# index = 1531
# def draw_network(mask,mask2,tube_network,index,main=False):
#     for tube in tube_network:
#         #print("tube "+str(len(tube)))
#         if len(tube)>0:
#             if isinstance(tube[0][0],type(np.array([]))) and not isinstance(tube[0][0][0],type(np.intc(0))):
#                 mask,mask2 = draw_network(mask,mask2,tube,index)
#             else:
#                 for point in tube:
#                     if main:
#                         mask[index][point[0][1]][point[0][0]] = 255
#                     else:
#                         mask2[index][point[0][1]][point[0][0]] = 255
#             index=index-1
#     return mask,mask2
import napari
# mask,mask2 = draw_network(mask,mask2,tube_network,index,True)
mask_list = networkManager.generate3DImages(image_array.shape)
viewer = napari.Viewer()
image_layer = viewer.add_image(image_array)
colorindex = 0
print(mask_list)
for mask in mask_list:
    image_layer = viewer.add_image(mask,colormap=list(cu.AVAILABLE_COLORMAPS.keys())[colorindex],blending="additive")
    colorindex+=1
#image_layer = viewer.add_image(mask2,colormap="blue",blending="additive")
#image_layer = viewer.add_image(last_search_mask,colormap="red",blending="additive")
# start the event loop and show the viewer
napari.run()
#print(sitk_image)


[array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0